# Plot EEG sensitivity to small dipole movements

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lfpykit.eegmegcalc import FourSphereVolumeConductor
from brainsignals.plotting_convention import mark_subplots, simplify_axes

head_colors = ["#ffb380", "#74abff", "#b3b3b3", "#c87137"]

radii = [89000., 90000., 95000., 100000.]  # (µm)
sigmas = [0.276, 1.65, 0.01, 0.465]  # (S/m)
p = np.array([[0.], [0.], [100.]]) # 1 timestep (nA µm)
human_cortex_thickness = 3 * 1e3  # 3 mm
r_elecs = np.array([[0, 0, radii[-1] - 1]]) # (µm)

dipole_depths = np.linspace(radii[0] - human_cortex_thickness, radii[0] - 100, 50)
lateral_x_shifts = np.linspace(0, 5000, 50)

default_depth_idx = np.argmin(np.abs(dipole_depths - (radii[0] - 1500)))
default_depth = dipole_depths[default_depth_idx]

def plot_four_sphere_model(ax, radii, dipole_loc=None):
    
    for i in range(4):
        ax.add_patch(plt.Circle((0, 0), radius=radii[-1 - i],
                                   color=head_colors[-1-i],
                                   fill=True, ec='k', lw=.1))
    if not dipole_loc is None:
        ax.plot(dipole_loc[0], dipole_loc[2], 'r.')
        
    # mark 4-sphere head model layers
    ax.text(-10500, radii[0] - 3500, 'brain', ha="right", rotation=4)
    ax.text(-10500, radii[1] - 2200, 'CSF', ha="right", rotation=4)
    ax.text(-10500, radii[2] - 4000, 'skull', ha="right", rotation=4)
    ax.text(-10500, radii[3] - 4000, 'scalp', ha="right", rotation=4)

        
def dipole_potential(elec_locs, dipole_pos, p):
    # Potential in infinite homogeneous medium
    r_ = elec_locs.T - dipole_pos
    V_e = 1000 * 1. / (4 * np.pi * sigmas[0]) * (np.dot(r_, p.T)
                    / np.linalg.norm(r_, axis=1) ** 3)
    return V_e
        

In [ ]:
depths_eeg_4s = np.zeros(len(dipole_depths))
depths_eeg_homog = np.zeros(len(dipole_depths))

lat_eeg_4s = np.zeros(len(lateral_x_shifts))
lat_eeg_homog = np.zeros(len(lateral_x_shifts))

sphere_model = FourSphereVolumeConductor(r_elecs, radii, sigmas)
for di, depth in enumerate(dipole_depths):
    dipole_loc = np.array([0., 0., depth])
    M = sphere_model.get_transformation_matrix(dipole_loc)  
    pot = 1e3 * M @ p # (uV)
    # Compare with potential from dipole in infinite homogeneous medium
    pot_homog = dipole_potential(r_elecs.T, dipole_loc, p[:, 0])
    
    depths_eeg_4s[di] = pot
    depths_eeg_homog[di] = pot_homog
    
for xi, lat_pos in enumerate(lateral_x_shifts):
    dipole_loc = np.array([lat_pos, 0., default_depth])
    M = sphere_model.get_transformation_matrix(dipole_loc)  
    pot = 1e3 * M @ p # (uV)
    # Compare with potential from dipole in infinite homogeneous medium
    pot_homog = dipole_potential(r_elecs.T, dipole_loc, p[:, 0])
    
    lat_eeg_4s[xi] = pot
    lat_eeg_homog[xi] = pot_homog


In [ ]:
fig = plt.figure(figsize=[6, 1.8])
fig.subplots_adjust(bottom=0.23, right=0.99, left=0.07, top=0.9, wspace=.5)
ax_geom = fig.add_subplot(131, frameon=False, aspect=1,
                         xticks=[], yticks=[],
                         xlim=[-10000, 10000],
                         ylim=[85000, 101000])
ax_depth = fig.add_subplot(132, xlabel="cortical depth (mm)", 
                         ylabel="relative change", yticks=[0.8, 0.9, 1.0, 1.1, 1.2])
ax_lat = fig.add_subplot(133, xlabel="lateral shift (mm)", 
                         ylabel="relative change", yticks=[0.8, 0.85, 0.9, 0.95, 1.0],
                        xticks=[0, 1, 2, 3, 4, 5])#ylim=[0, 0.1])
plot_four_sphere_model(ax_geom, radii)

ax_geom.plot(r_elecs[0, 0], r_elecs[0, 2], 'o', c='k')
ax_geom.plot(np.zeros(len(dipole_depths)), dipole_depths, 'r')
ax_geom.plot(lateral_x_shifts, np.ones(len(lateral_x_shifts)) * default_depth, 'b')

ax_depth.plot(-(dipole_depths - radii[0]) / 1000, depths_eeg_4s / depths_eeg_4s[default_depth_idx], c='r')
ax_depth.plot(-(dipole_depths - radii[0]) / 1000, depths_eeg_homog/ depths_eeg_homog[default_depth_idx], 'r--')

ax_lat.plot(lateral_x_shifts / 1000, lat_eeg_4s / lat_eeg_4s[0], c='b')
ax_lat.plot(lateral_x_shifts / 1000, lat_eeg_homog / lat_eeg_homog[0], 'b--')

l1, = ax_depth.plot([], 'k-')
l2, = ax_depth.plot([], 'k--')

fig.legend([l2, l1], ["infinite homogeneous", "four-sphere", ], 
           frameon=False, ncol=1, loc=(0.03, 0.0))
simplify_axes([ax_depth, ax_lat])
mark_subplots(fig.axes, ypos=1.05, xpos=-0.05)
plt.savefig("eeg_sensitivity_to_dipole_loc.pdf")